In [2]:
import pickle
import numpy as np
import pandas as pd
import glob
import nltk
import unidecode
import contractions
from nltk.corpus import stopwords

with open('partial_fit_model_no_stop', 'rb') as files:
    l = pickle.load(files)
    
with open('count_vectorizer_no_stop', 'rb') as f:
    t = pickle.load(f)    

with open('label_encoder_no_stop', 'rb') as fil:
    v = pickle.load(fil)

In [3]:
df_all = pd.read_csv('codemix_data.csv')

In [4]:
l.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64)

In [5]:
stop = stopwords.words('english')

In [6]:
def expand_contractions(text):
    expanded_text = contractions.fix(text) 
    return expanded_text

In [7]:
contracted_stop = []
for i in stop:
    contracted_stop.append(expand_contractions(i))

In [8]:
contracted_stw = []
for i in contracted_stop:
    contracted_stw.append(i)

In [9]:
all_stops = stop + contracted_stw

In [10]:
len(all_stops) 

358

In [11]:
from bs4 import BeautifulSoup
import requests

In [12]:
page = requests.get('https://www.ef-australia.com.au/english-resources/english-vocabulary/top-3000-words/')
soup = BeautifulSoup(page.text, 'lxml')

In [13]:
eng_words = []
for i in soup.findAll('p')[11:]:
    for j in i:
        if j!=' ' and j!='':
            eng_words.append(j.text)

In [14]:
eng_words = eng_words + all_stops

In [15]:
len(eng_words)

6357

In [16]:
english_s = []

In [17]:
for word in eng_words:
    english_s.append(f'{word}ing')

In [18]:
for word in eng_words:
    english_s.append(f'{word}s')

In [19]:
for word in eng_words:
    english_s.append(f'{word}es')

In [20]:
for word in eng_words:
    english_s.append(f'{word}ed')

In [21]:
eng_words = english_s + eng_words

In [22]:
len(eng_words)

31785

In [23]:
def predictFind_codemix(text): 
    
    global eng_words
    english_words = []
    text_for_pred = []
    for token in text.lower().split():
        if token in eng_words:
            english_words.append(token)
    
    text_for_pred = str([token for token in text.lower().split() if token not in english_words])
    
    global l, t, v
    x = t.transform([text_for_pred]).toarray() 
    lang = l.predict(x) 
    lang_per = l.predict_proba(x) 
    lang = v.inverse_transform(lang)
    print(f'''The text is in {str(lang[0])} with {round(float(lang_per.max()*100), 1)}% confidence \n\n''') 
    
    lang_names = list(v.inverse_transform(l.classes_)) 
    probas_list = []
    for i in lang_per:
        for j in i:
            probas_list.append(j)

    proba_table = pd.DataFrame({'Language': lang_names,
                        'Probability': probas_list}).sort_values(by='Probability',
                                                           ascending=False).reset_index(drop=True)
                
    if len(english_words)>0 and str(proba_table['Language'].loc[0])!='English':
        with open(f'''{proba_table['Language'].loc[0]}-English-codemix.txt''', 'a') as file:
            file.write(f'{text}\n')
            file.close()  
            
    elif len(english_words)>0 and str(proba_table['Language'].loc[0])=='English' and proba_table['Probability'].loc[0]<0.13:
        with open(f'''Unknown-dialect-codemix.txt''', 'a') as file:
            file.write(f'{text}\n')
            file.close()  
            
    elif len(english_words)>0 and str(proba_table['Language'].loc[0])=='English' and proba_table['Probability'].loc[0]>0.13:
        pass

In [24]:
len(df_all)

316942

In [330]:
for text in df_all['sentence'].values:
    predictFind_codemix(text)

The text is in Telugu with 100.0% confidence 


The text is in Telugu with 20.8% confidence 


The text is in Kannada with 13.1% confidence 


The text is in Kannada with 15.0% confidence 


The text is in English with 22.6% confidence 


The text is in Hindi with 74.9% confidence 


The text is in English with 22.6% confidence 


The text is in English with 42.2% confidence 


The text is in English with 42.2% confidence 


The text is in English with 13.0% confidence 


The text is in Telugu with 45.6% confidence 


The text is in Hindi with 47.5% confidence 


The text is in Telugu with 79.7% confidence 


The text is in Telugu with 42.9% confidence 


The text is in Kannada with 32.5% confidence 


The text is in English with 13.0% confidence 


The text is in English with 13.0% confidence 


The text is in English with 13.0% confidence 


The text is in English with 36.1% confidence 


The text is in Gujarati with 19.4% confidence 


The text is in English with 25.7% confidence 

